In [1]:
import argparse
import os
import numpy as np
import json
from model.eval_voc import parse_voc_annotation
from model.eval_gen import BatchGenerator
from model.eval_utils import normalize, _sigmoid,makedirs,correct_yolo_boxes,do_nms,decode_netout,preprocess_input
from model.eval_utils import get_yolo_boxes,compute_overlap,compute_ap,_softmax
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import load_model
from model.yolo3 import yolo_body, tiny_yolo_body
from model.mobilenet import mobilenetv2_yolo_body
from keras.layers import Input

Using TensorFlow backend.


In [2]:
###############################
    #   Create the validation generator
    ###############################  
model_labels = [ "aeroplane", "bicycle", "bird","boat","bottle", "bus","car", "cat","chair", "cow", "diningtable", "dog","horse","motorbike","person", "pottedplant", "sheep","sofa", "train","tvmonitor"]
valid_ints, labels = parse_voc_annotation(
        "VOCdevkit/1VOC/Annotations/", 
        "VOCdevkit/1VOC/JPEGImages/", 
        "voc1.pkl",
        model_labels
)

labels = labels.keys() if len(model_labels) == 0 else model_labels
labels = sorted(labels)

model_anchors = [ 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326 ]
print("valid generator")
valid_generator = BatchGenerator(
        instances           = valid_ints, 
        anchors             = model_anchors,   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = 0,
        batch_size          = 2,
        min_net_size        = 288,
        max_net_size        = 448,   
        shuffle             = True, 
        jitter              = 0.0, 
        norm                = normalize
    )
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

valid generator


In [3]:
infer_model = yolo_body(Input(shape=(None,None,3)), 3 , 20)
infer_model.load_weights("model_data/trained_weights_final.h5")

In [4]:
np.set_printoptions(threshold=np.inf)

In [24]:
#evaluate(infer_model, valid_generator)
model = infer_model 
generator = valid_generator 
iou_threshold=0.5
obj_thresh=0.5
nms_thresh=0.45
net_h=416
net_w=416
save_path=None

# gather all detections and annotations
all_detections     = [[None for i in range(generator.num_classes())] for j in range(generator.size()) ] # array of none
all_annotations    = [[None for i in range(generator.num_classes())] for j in range(generator.size()) ]

generator.size()

1

In [25]:
   # gather all detections and annotations
for i in range(generator.size()) :
        print(i)
        raw_image = [generator.load_image(i)]
        #print("generator = "+str(i) +" from " + str(generator.size()) )
        # make the boxes and the labels
        pred_boxes = get_yolo_boxes(model, raw_image, net_h, net_w, generator.get_anchors(), obj_thresh, nms_thresh)[0]
        
        #print("generator anchor = "+str(generator.get_anchors()) )
        score = np.array([box.get_score() for box in pred_boxes])
        print( score.shape )
        pred_labels = np.array([box.label for box in pred_boxes])        
        
        if len(pred_boxes) > 0:
            pred_boxes = np.array([[box.xmin, box.ymin, box.xmax, box.ymax, box.get_score()] for box in pred_boxes]) 
            print(pred_boxes.shape)
        else:
            pred_boxes = np.array([[]])  
        
        # sort the boxes and the labels according to scores
        score_sort = np.argsort(-score)
        pred_labels = pred_labels[score_sort]
        pred_boxes  = pred_boxes[score_sort]
        
        # copy detections to all_detections
        for label in range(generator.num_classes()):
            #print("copy detections to all_detections  label "+str(label) )
            all_detections[i][label] = pred_boxes[pred_labels == label, :]

        annotations = generator.load_annotation(i)
        
        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            #print("copy detections to all_annotations  label "+str(label) )
            all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()


0
(16,)
(16, 5)


In [26]:
len(all_detections )

1

In [30]:
# compute mAP by comparing all detections and all annotations
average_precisions = {}
for label in  range(generator.num_classes()) :
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0

        #print("get average precisions every label"+str(label) )
       
        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []
            
            #print("group all detection "+str(generator) )

            for d in detections:
                print("detection group"+ str(d) )
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    #print( "true_positive = {}".format(true_positives) )
                    continue
                
                #print(annotations.shape[0])
                
                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations)
                print(overlaps)
                assigned_annotation = np.argmax(overlaps, axis=1)
                print(assigned_annotation)
                max_overlap         = overlaps[0, assigned_annotation]

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                    
                   # print( "true_positive = {}".format(true_positives) )
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                   # print( "true_positive = {}".format(true_positives) )
                    

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            print("no AP" + str(label) )
            continue

        # sort by score
        print("Scores{}".format(scores) )
        print( "a true_positive = {}".format(true_positives) )
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        print( "b true_positive = {}".format(true_positives) )

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)
        print( "c true_positive = {}".format(true_positives) )

        # compute recall and precision
        recall    = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        print( "num anno = {}".format(num_annotations) )
        print( "recall = {}".format(recall) )
        print( "precision = {}".format(precision) )
        print( "cancer = {}".format(np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)) )
        # compute average precision
        average_precision  = compute_ap(recall, precision)  
        print( "average_precisions = {}".format(average_precisions) )
        average_precisions[label] = average_precision
        
average_precisions

detection group[ 16.  -2. 236. 444.   0.]
detection group[  9. -10. 243. 456.   0.]
detection group[-19.   5. 226. 479.   0.]
detection group[ 17.   0. 232. 483.   0.]
detection group[ 27.  14. 276. 472.   0.]
detection group[ -2.  61. 240. 490.   0.]
detection group[ -2. 178. 196. 497.   0.]
detection group[  2. 185. 243. 488.   0.]
detection group[215. 195. 350. 485.   0.]
detection group[  9. 208. 181. 506.   0.]
detection group[244. 200. 365. 508.   0.]
detection group[ 48. 256. 209. 516.   0.]
no AP0
detection group[ 27.         224.         223.         498.           0.99978298]
[[0.80941815 0.         0.26647936]]
[0]
detection group[229.         202.         340.         503.           0.84637016]
[[0.04654621 0.87156924 0.        ]]
[1]
Scores[0.99978298 0.84637016]
a true_positive = [1. 1.]
b true_positive = [1. 1.]
c true_positive = [1. 2.]
num anno = 3.0
recall = [0.33333333 0.66666667]
precision = [1. 1.]
cancer = [1. 2.]
average_precisions = {0: 0}
no AP2
no AP3
no AP4
n

{0: 0,
 1: 0.6666666666666666,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0.6666666666666666,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0}

In [28]:
# print the score
for label, average_precision in average_precisions.items():
    print(labels[label] + ': {:.4f}'.format(average_precision))
print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))  

aeroplane: 0.0000
bicycle: 0.6667
bird: 0.0000
boat: 0.0000
bottle: 0.0000
bus: 0.0000
car: 0.0000
cat: 0.0000
chair: 0.0000
cow: 0.0000
diningtable: 0.0000
dog: 0.0000
horse: 0.0000
motorbike: 0.0000
person: 0.6667
pottedplant: 0.0000
sheep: 0.0000
sofa: 0.0000
train: 0.0000
tvmonitor: 0.0000
mAP: 0.0667
